<a href="https://colab.research.google.com/github/Carvas91/data_analysis_portfolio/blob/main/Coordinates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

For this project, I utilized the Brampton, Ontario business directory database to create an HTML map using the Folium library. This was done to provide my manager with potential leads. I clustered the businesses based on their proximity to facilitate easier visit planning.

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None) #all columns will be displayed without truncation

In [7]:
#url = 'https://geohub.brampton.ca/datasets/brampton::brampton-business-directory/explore'
df = pd.read_csv('/content/drive/MyDrive/data/Brampton_Business_Directory.csv')
df.columns

Index(['X', 'Y', 'OBJECTID', 'COMPANY_NAME', 'OPERATIONAL', 'STREET_NUM',
       'STREET_NAME', 'STREET_TYPE', 'STREET_DIRECTION', 'UNIT',
       'BUSINESS_FULL_ADDRESS', 'CITY', 'PROVINCE', 'POSTAL_CODE', 'PHONE',
       'FAX', 'WEBURL', 'DATE_EST_IN_CITY', 'STARTED_IN_CITY',
       'HEAD_OFFICE_LOCATION', 'TOTAL_EMPLOYEE', 'SHIFTS', 'SHIFT_NUMBER',
       'GFA_SQUARE_FEET', 'GFA_SQUARE_METER', 'NAIC_2', 'NAIC_3', 'NAIC_4',
       'NAIC_6', 'PRODUCT_DESC', 'EXPORTS', 'EXPORT_PERCENTAGE',
       'EXPORT_COUNTRY1', 'EXPORT_COUNTRY2', 'EXPORT_COUNTRY3', 'UPDATE_DATE',
       'UPDATED_BY', 'NAICS_DETAIL', 'CREATED_BY', 'CREATE_DATE', 'FACEBOOK',
       'TWITTER', 'LINKEDIN', 'YOUTUBE', 'INSTAGRAM', 'GLOBALID'],
      dtype='object')

In [8]:
print(df.index[df['STREET_NAME'] == 'HURONARIO'])
print(df.index[df['STREET_NAME'] == 'HURON'])


Int64Index([9429], dtype='int64')
Int64Index([4888], dtype='int64')


In [ ]:
df.loc[9429, 'STREET_NAME'] = 'HURONTARIO' #here I corrected a cell that wasn't written correctly
df.loc[4888, 'STREET_NAME'] = 'HURONTARIO'

In [ ]:
df.to_csv('Brampton_Business_Directory_updated.csv', index=False) #this is the updated dataset


In [9]:
#Here, I utilized a function to transform the coordinates in the DataFrame into standard format coordinates.
from pyproj import Proj, transform

proj_from = Proj('epsg:3857')  # Web Mercator
proj_to = Proj('epsg:4326')    # WGS 84

# Function to transform a single coordinate pair
def transform_coord(x, y):
    return transform(proj_from, proj_to, x, y)

# Applying the transformation to each row in the DataFrame
df['latitude'], df['longitude'] = zip(*df.apply(lambda row: transform_coord(row['X'], row['Y']), axis=1))


<ipython-input-9-2686c3faa426>:9: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  return transform(proj_from, proj_to, x, y)


In [11]:
df[['X', 'Y', 'COMPANY_NAME', 'longitude', 'latitude','STREET_NAME']].head(5)

,X,Y,COMPANY_NAME,longitude,latitude,STREET_NAME
0,-8.879769e+06,5.421624e+06,ABC FIRST AID AND CPR TRAINING SERVICES,-79.768324,43.716176,RUSTHALL
1,-8.875687e+06,5.413618e+06,DR. JAGTARAN SINGH DHALIWAL,-79.731652,43.664170,HURONTARIO
2,-8.873948e+06,5.428276e+06,UC BABY,-79.716034,43.759351,COTTRELLE
3,-8.869585e+06,5.421456e+06,APPLIANCE KING,-79.676839,43.715084,STEELES
4,-8.879045e+06,5.416237e+06,CHAMPAGNE AND ASSOCIATES,-79.761822,43.681189,ELLIOTT


In [12]:

# This is our reference point
ref_latitude = 43.666071907975784
ref_longitude = -79.73871781748225

# Function to calculate distance using Haversine formula
def haversine(lon1, lat1, lon2, lat2):
    from math import radians, cos, sin, asin, sqrt
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    r = 6371  # Radius of earth in kilometers
    return c * r

# Initialize an empty list to store the distances
distances = []

# Iterate over each row in the DataFrame
for _, row in df.iterrows():
    distance = haversine(ref_longitude, ref_latitude, row['longitude'], row['latitude'])
    distances.append(distance)

# Add the distances as a new column to the DataFrame
df['distance_to_ref_km'] = distances

# Now df includes the distances from each point to the reference point


In [13]:
df[['X', 'Y', 'COMPANY_NAME', 'longitude', 'latitude','STREET_NAME', 'distance_to_ref_km' ]].sample(5)


,X,Y,COMPANY_NAME,longitude,latitude,STREET_NAME,distance_to_ref_km
137,-8.877425e+06,5.428945e+06,SPRINGDALE CONSULTING,-79.747270,43.763690,COBBLESTONE,10.876386
2127,-8.878092e+06,5.418018e+06,DR. A. R KIDY,-79.753255,43.692757,QUEEN,3.189208
7567,-8.867846e+06,5.430194e+06,ISLAND GROVE ROTI AND BAR,-79.661214,43.771791,EBENEZER,13.303543
1323,-8.878116e+06,5.417538e+06,UPHILL CARDIAC WELLNESS CENTRE,-79.753472,43.689641,CENTRE,2.876861
6732,-8.873952e+06,5.428271e+06,OSLER VASCULAR & VEIN,-79.716066,43.759316,COTTRELLE,10.526878


In [14]:
# Calculate the average distance for each STREET_NAME
average_distances = df.groupby('STREET_NAME')['distance_to_ref_km'].mean().reset_index()
average_distances = average_distances.rename(columns={'distance_to_ref_km': 'avg_distance_to_ref_km'})

# Merge the average distance back into the original DataFrame
df = pd.merge(df, average_distances, on='STREET_NAME', how='left')

# Now df includes the average distance to reference point for each STREET_NAME


In [15]:
# Save the DataFrame to a CSV file
df.to_csv('updated_data.csv', index=False)

# The 'index=False' parameter is used to tell pandas not to write row names (indices)


In [16]:
import folium
from folium.plugins import MarkerCluster

# Initial coordinates for Brampton city center
brampton_center = [43.7315, -79.7624]

# Creating a map centered around Brampton
map_brampton = folium.Map(location=brampton_center, zoom_start=12)

# Initializing a marker cluster
marker_cluster = MarkerCluster().add_to(map_brampton)

# Adding each business location to the marker cluster
for idx, row in df.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=(f"{row['COMPANY_NAME']}\n"
               f"Address: {row['BUSINESS_FULL_ADDRESS']}\n"
               f"Distance to Ref: {row['distance_to_ref_km']:.2f} km"),
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(marker_cluster)

# Displaying the map
map_brampton.save('brampton_business_map.html')
